# 🔍 Exploration des données - Détection de Fraude

**Objectif** : Comprendre la structure des données et identifier les patterns de fraude

Petite note pour moi : Ce projet me rappelle un peu le travail que j'ai fait chez Groupama sur la prédiction de satisfaction client. Même logique de déséquilibre de classes et d'importance des coûts métier.

**Dataset** : Credit Card Fraud Detection
- Transactions par carte bancaire
- Features anonymisées (PCA)
- Très déséquilibré (comme d'hab avec la fraude...)


In [ ]:
# Imports classiques
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Config pour les graphiques
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Imports OK")


## 1. Chargement des données

Bon, voyons ce qu'on a dans ce dataset...


In [ ]:
# Charger les données
data_path = Path('../data/raw/creditcard.csv')
df = pd.read_csv(data_path)

print(f"📊 Shape: {df.shape}")
print(f"💾 Mémoire: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
df.head()


In [ ]:
# Infos générales
df.info()


## 2. Analyse du déséquilibre

Le gros problème avec la détection de fraude c'est toujours le déséquilibre...
Voyons combien de fraudes on a vraiment.


In [ ]:
# Distribution des classes
class_counts = df['Class'].value_counts()
fraud_rate = (class_counts[1] / len(df)) * 100

print(f"Transactions normales: {class_counts[0]:,}")
print(f"Fraudes: {class_counts[1]:,}")
print(f"Taux de fraude: {fraud_rate:.3f}%")
print(f"\n💡 Ratio: 1 fraude pour {class_counts[0] // class_counts[1]} transactions normales")

# Viz rapide
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Barplot
class_counts.plot(kind='bar', ax=ax[0], color=['#2ecc71', '#e74c3c'])
ax[0].set_title('Distribution des Classes', fontsize=12, fontweight='bold')
ax[0].set_xlabel('Classe (0=Normal, 1=Fraude)')
ax[0].set_ylabel('Nombre de transactions')
ax[0].set_xticklabels(['Normal', 'Fraude'], rotation=0)

# Pie chart
colors = ['#2ecc71', '#e74c3c']
ax[1].pie(class_counts, labels=['Normal', 'Fraude'], autopct='%1.2f%%', 
          colors=colors, startangle=90)
ax[1].set_title('Proportion des Classes', fontsize=12, fontweight='bold')

plt.tight_layout()
plt.show()

# Note: Va falloir gérer ce déséquilibre avec SMOTE ou autre...


## 3. Analyse des montants

Les montants des transactions peuvent être un bon indicateur. Les fraudes sont-elles plutôt sur des petits ou gros montants ?


In [ ]:
# Stats sur les montants
print("=== Statistiques des montants ===\n")
print("Toutes transactions:")
print(df['Amount'].describe())
print("\n" + "="*40)
print("\nTransactions normales:")
print(df[df['Class']==0]['Amount'].describe())
print("\n" + "="*40)
print("\nFraudes:")
print(df[df['Class']==1]['Amount'].describe())

# Visualisation
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Distribution des montants (toutes transactions)
df['Amount'].hist(bins=50, ax=axes[0], color='skyblue', edgecolor='black')
axes[0].set_title('Distribution des montants (Toutes)')
axes[0].set_xlabel('Montant (€)')
axes[0].set_ylabel('Fréquence')
axes[0].set_yscale('log')

# Boxplot comparaison
df.boxplot(column='Amount', by='Class', ax=axes[1])
axes[1].set_title('Montants : Normal vs Fraude')
axes[1].set_xlabel('Classe')
axes[1].set_ylabel('Montant (€)')

# Distribution pour fraudes uniquement (pour mieux voir)
df[df['Class']==1]['Amount'].hist(bins=30, ax=axes[2], color='#e74c3c', edgecolor='black')
axes[2].set_title('Distribution des montants (Fraudes uniquement)')
axes[2].set_xlabel('Montant (€)')
axes[2].set_ylabel('Fréquence')

plt.tight_layout()
plt.show()

# Observation perso : Les fraudes semblent avoir des montants assez variables. 
# Pas de pattern évident juste avec Amount. Les features V1-V28 vont être importantes.
